## 3.3 线性回归的简洁实现

### 3.3.1 生成数据集 

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2 读取数据集

In [5]:
def load_array(data_arrays, batch_size, is_true=True):
    #构造一个PyTorch数据迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_true)

In [11]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[-0.9067,  0.9007],
         [-0.0034, -0.7338],
         [ 0.3460, -1.1590],
         [-0.0366, -0.0084],
         [ 1.4603, -1.0409],
         [ 1.2620, -0.9941],
         [-2.6212, -1.5690],
         [ 0.4731, -0.2865],
         [ 0.2694,  0.0105],
         [ 0.5233, -0.0115]]),
 tensor([[-0.6730],
         [ 6.6887],
         [ 8.8247],
         [ 4.1592],
         [10.6670],
         [10.1016],
         [ 4.2900],
         [ 6.1252],
         [ 4.6966],
         [ 5.2889]])]

### 3.3.3 定义模型

In [12]:
# nn是神经网络的缩写
from torch import nn
# Sequential实例将数据传入到第一层，然后将第一层的输出作为第二层的输入
# Linear中的参数：第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1
net = nn.Sequential(nn.Linear(2, 1))


### 3.3.4 初始化模型参数 

In [14]:
# net[0]选择的是网络中的第一个图层
# 后使用weight.data和bias.data方法访问参数。我们还可以使用替换方法normal_和fill_来重写参数值
net[0].weight.data.normal_(0, 0.01)
net[0].weight.data.fill_(0)

tensor([[0., 0.]])

### 3.3.5 定义损失函数 

In [15]:
loss = nn.MSELoss()

### 3.3.6 定义优化算法

In [16]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 3.3.7 训练
步骤：
• 通过调用net(X)生成预测并计算损失l（前向传播）。
• 通过进行反向传播来计算梯度。
• 通过调用优化器来更新模型参数。

In [18]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        #计算损失，预测值与真实值之间的误差
        l = loss(net(X) ,y)
        #梯度归零
        trainer.zero_grad()
        #计算梯度
        l.backward()
        #更新梯度
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch 1, loss 0.000099
epoch 2, loss 0.000099
epoch 3, loss 0.000099
w的估计误差： tensor([-0.0005,  0.0006])
b的估计误差： tensor([0.0002])
